# 픽셀 노말라이제이션
* Progressive GAN 에서 사용한 pixel normalization 레이어를 만들어보자.

In [1]:
import tensorflow as tf
from tensorflow.keras import models, layers, optimizers, losses, metrics
import numpy as np
import time

In [2]:
tf.__version__

'2.1.0'

In [3]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

## 하이퍼파라미터

In [4]:
BATCH_SIZE = 2**5
EPOCHS = 10
print(BATCH_SIZE, EPOCHS)

32 10


## 데이터

In [5]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [6]:
X_train = np.array(X_train/np.max(X_train), dtype=np.float32)
X_test = np.array(X_test/np.max(X_test), dtype=np.float32)

In [7]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(50000, 32, 32, 3) (50000, 1)
(10000, 32, 32, 3) (10000, 1)


In [8]:
traindataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(BATCH_SIZE).shuffle(len(X_train))
testdataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(BATCH_SIZE)

## 픽셀 노말라이제이션
* 알렉스 넷(Krizhevsky et al., 2012)에서 제안한 local response normalization을 아래 수식으로 변형하여 사용하고 pixel normalization 이라 이름 붙임

$$
b_{x,y} = {a_{x, y} \over \sqrt{{1\over N} \sum_{j=0}^{N-1} \bigg( a^j_{x,y} \bigg)^2 + \epsilon}}
$$

* 이 때, $\epsilon=10^-8$
* $N$ : feature map 개수
* $a_{x,y}, b_{x,y}$ : 픽셀 $x,y$에 대한 원래 피처벡터와 노말라이즈드 된 피처벡터
* 참고로 알렉스넷의 local respose normalizaiton 은 ```tf.nn.local_response_normalization```가 있음

In [38]:
class PixelNormalization(layers.Layer):
    def __init__(self, epsilon=1e-8):
        super(PixelNormalization, self).__init__()
        self.epsilon = epsilon
        
    def call(self, inputs):
        x = inputs
        return x * tf.math.rsqrt(tf.reduce_mean(tf.square(x), axis=None, keepdims=True) + self.epsilon)

## 얕은 모델에 적용

In [10]:
class Model(models.Model):
    def __init__(self):
        super(Model, self).__init__()
        self.model = models.Sequential([
            layers.Conv2D(64, (3,3), (1,1), activation=tf.nn.relu),
            layers.Conv2D(64, (3,3), (1,1), activation=tf.nn.relu),
            layers.MaxPool2D((2,2)),
            layers.Conv2D(64, (3,3), (1,1), activation=tf.nn.relu),
            layers.Conv2D(64, (3,3), (1,1), activation=tf.nn.relu),
            layers.MaxPool2D((2,2)),
            layers.Flatten(),
            layers.Dense(128, activation=tf.nn.relu),
            layers.Dense(10, activation=tf.nn.softmax)
        ])
        
    def call(self, inputs, training=False):
        outputs = self.model(inputs, training=training)
        return outputs
    
model = Model()
model(X_train[:1]).shape

TensorShape([1, 10])

In [11]:
sce = losses.SparseCategoricalCrossentropy()
adam = optimizers.Adam(1e-3)
loss = metrics.Mean()

In [12]:
def train_step(inputs):
    _x, _y = inputs
    
    with tf.GradientTape() as t:
        y_pred = model(_x, True)
        _loss = sce(_y, y_pred)
        
    grads = t.gradient(_loss, model.trainable_variables)
    adam.apply_gradients(list(zip(grads, model.trainable_variables)))
    loss.update_state(_loss)

In [13]:
for e in range(EPOCHS):
    start = time.time()
    for inputs in traindataset:
        train_step(inputs)
        
    print(f"{e+1}/{EPOCHS}\tloss = {loss.result():.8f}\t{time.time()-start:.2f}sec/epoch")
    loss.reset_states()

1/10	loss = 1.43167722	32.09sec/epoch
2/10	loss = 0.99789804	31.97sec/epoch
3/10	loss = 0.81246853	31.60sec/epoch
4/10	loss = 0.70031887	31.34sec/epoch
5/10	loss = 0.61313117	31.92sec/epoch
6/10	loss = 0.53898168	31.41sec/epoch
7/10	loss = 0.47794420	31.75sec/epoch
8/10	loss = 0.41353273	31.61sec/epoch
9/10	loss = 0.36280429	31.26sec/epoch
10/10	loss = 0.31180003	31.09sec/epoch


## PN 없이 성능 확인

In [21]:
sca = metrics.SparseCategoricalAccuracy()
acc = metrics.Mean()

for inputs in testdataset:
    _x, _y = inputs
    pred = model(_x)
    acc.update_state(sca(_y, pred))
    
print(acc.result().numpy())

0.74453866


## PN 넣은 모델 만들기

In [39]:
class PN_Model(models.Model):
    def __init__(self):
        super(PN_Model, self).__init__()
        self.model = models.Sequential([
            layers.Conv2D(64, (3,3), (1,1), activation=tf.nn.relu),
            PixelNormalization(),
            layers.Conv2D(64, (3,3), (1,1), activation=tf.nn.relu),
            PixelNormalization(),
            layers.MaxPool2D((2,2)),
            layers.Conv2D(64, (3,3), (1,1), activation=tf.nn.relu),
            PixelNormalization(),
            layers.Conv2D(64, (3,3), (1,1), activation=tf.nn.relu),
            PixelNormalization(),
            layers.MaxPool2D((2,2)),
            layers.Flatten(),
            layers.Dense(128, activation=tf.nn.relu),
            layers.Dense(10, activation=tf.nn.softmax)
        ])
        
    def call(self, inputs, training=False):
        outputs = self.model(inputs, training=training)
        return outputs
    
pnmodel = PN_Model()
pnmodel(X_train[:1]).shape

TensorShape([1, 10])

In [42]:
def pntrain_step(inputs):
    _x, _y = inputs
    
    with tf.GradientTape() as t:
        y_pred = pnmodel(_x, True)
        _loss = sce(_y, y_pred)
        
    grads = t.gradient(_loss, pnmodel.trainable_variables)
    adam.apply_gradients(list(zip(grads, pnmodel.trainable_variables)))
    loss.update_state(_loss)

In [43]:
for e in range(EPOCHS):
    start = time.time()
    for inputs in traindataset:
        pntrain_step(inputs)
        
    print(f"{e+1}/{EPOCHS}\tloss = {loss.result():.8f}\t{time.time()-start:.2f}sec/epoch")
    loss.reset_states()

1/10	loss = 1.89180040	49.31sec/epoch
2/10	loss = 1.50700188	49.61sec/epoch
3/10	loss = 1.38040340	49.14sec/epoch
4/10	loss = 1.28476465	48.94sec/epoch
5/10	loss = 1.19977808	49.37sec/epoch
6/10	loss = 1.12631416	62.68sec/epoch
7/10	loss = 1.06056011	48.67sec/epoch
8/10	loss = 1.00555885	49.92sec/epoch
9/10	loss = 0.94893080	48.11sec/epoch
10/10	loss = 0.90535581	50.21sec/epoch


## PN 넣고 성능확인

In [45]:
sca = metrics.SparseCategoricalAccuracy()
acc = metrics.Mean()

for inputs in testdataset:
    _x, _y = inputs
    pred = pnmodel(_x)
    acc.update_state(sca(_y, pred))
    
print(acc.result().numpy())

0.65336835


* 얌전히 GAN에나 쓰자.